In [1]:
!pip install pdfminer.six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 5.3 MB/s 
     |████████████████████████████████| 4.0 MB 48.7 MB/s 


In [32]:
!wget https://dm.takaratomy.co.jp/img/dm_rule_20220408.pdf

--2022-06-19 07:49:42--  https://dm.takaratomy.co.jp/img/dm_rule_20220408.pdf
Resolving dm.takaratomy.co.jp (dm.takaratomy.co.jp)... 125.29.34.172
Connecting to dm.takaratomy.co.jp (dm.takaratomy.co.jp)|125.29.34.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1275284 (1.2M) [application/pdf]
Saving to: ‘dm_rule_20220408.pdf’

dm_rule_20220408.pd 100%[===================>]   1.22M   252KB/s    in 5.1s    

2022-06-19 07:49:48 (242 KB/s) - ‘dm_rule_20220408.pdf’ saved [1275284/1275284]



#pdfからテキストデータの抽出 

参考URL:https://liquidjumper.com/programming/python/python_pdf_import_text_pdfminer#subheadline4


In [ ]:
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
from io import StringIO

pdf_file_path = "dm_rule_20220408.pdf"#ダウンロードしてきたpdfの名前に変更

with open(pdf_file_path , "rb") as pdf_file: #ファイルオブジェクトを受け取り、変数「pdf_file」に代入。
    output = StringIO() #コンソールに出力されたテキストを取得するため、IOクラス「StringIO」使用
    resource_manager = PDFResourceManager()
    laparams = LAParams()#レイアウトの変更がなければデフォルトのままで 
    text_converter = TextConverter(resource_manager, output, laparams=laparams)
    page_interpreter = PDFPageInterpreter(resource_manager, text_converter)

    for i_page in PDFPage.get_pages(pdf_file): #1ベージずづ処理 
        page_interpreter.process_page(i_page)

    output_text = output.getvalue()
    output.close()
    text_converter.close()
data_shaped=[]



print(output_text)

#データ整形

In [23]:
import re
texts=[]
for st in output_text.replace("\x0c","").split("\n"):#文字の空白を削除しつつ、改行ごとに処理
  if len(st)>2:
    if not re.match(r'^[0-9]+\.', st):
      if len(texts)!=0:
        texts[-1]+=st
    else:
      texts.append(st)
ids=[re.search(r"^[0-9]+(\.[0-9]?[0-9])*[a-z]*\.*",st).replace(" ","") for st in texts]#ルールID
rules=[re.sub(r"^[0-9]+(\.[0-9]?[0-9])*[a-z]*\.*","",st).replace(" ","") for st in texts ]#ルール内容




In [29]:
import pandas as pd
df=pd.DataFrame()
df["rule_dict_id"]=ids
df["rule"]=rules

In [28]:
pdf_file_path[:-4]

'dm_rule_20220408'

In [30]:
df.to_csv(pdf_file_path[:-4]+".csv",index=None)#csv形式で保存